Basic file reading and data exploration

In [1]:
import h5py
import numpy as np
import math

# fname = 'd4/20220302_131424.hdf5'
# fname = '20220408_191041.hdf5'
fname = '20220408_210049.hdf5'
f = h5py.File(fname, 'r')
print('File keys', f.keys())
print(type(f))

File keys <KeysViewHDF5 ['burr_change', 'data', 'force', 'metadata', 'voxels_removed']>
<class 'h5py._hl.files.File'>


In [2]:
data = f['data']
print('Data keys', data.keys())
# print((data['depth'][()]).shape)
# print((data['l_img'][()]).shape)
# print((data['pose_main_camera'][()]).shape)
print((data['pose_mastoidectomy_drill'][()]).shape)
# print((data['r_img'][()]).shape)
print((data['segm'][()]).shape)
print((data['time'][()]).shape)

# Not necessary for our purposes

# b_cg = f['burr_change']
# print('Burr Change keys', b_cg.keys())

# meta = f['metadata']
# print('Metadata keys', meta.keys())
# print((meta['README'][()]))

force = f['force']
print('Force keys', force.keys())
print((force['time_stamp'][()]).shape)
print((force['wrench'][()]).shape)

v_rm = f['voxels_removed']
print('Voxels Removed keys', v_rm.keys())
print((v_rm['time_stamp'][()]).shape)
print((v_rm['voxel_color'][()]).shape)
print((v_rm['voxel_removed'][()]).shape)

# # Time differences between either time stamps unequal

# print((data['time'][1]) - (data['time'][0]))
# print((data['time'][435]) - (data['time'][434]))
# print((v_rm['time_stamp'][1]) - (v_rm['time_stamp'][0]))
# print((v_rm['time_stamp'][435]) - (v_rm['time_stamp'][434]))

# print(min(data['time'][()]), max(data['time'][()]))

print(min(data['time'][()]) == min(data['time'][()]))

Data keys <KeysViewHDF5 ['depth', 'l_img', 'pose_main_camera', 'pose_mastoidectomy_drill', 'r_img', 'segm', 'time']>
(127, 7)
(127, 480, 640, 3)
(127,)
Force keys <KeysViewHDF5 ['time_stamp', 'wrench']>
(36771,)
(36771, 3)
Voxels Removed keys <KeysViewHDF5 ['time_stamp', 'voxel_color', 'voxel_removed']>
(13657,)
(13657, 4)
(13657, 3)
True


Function implementation

In [3]:
def get_strokes(stream: np.ndarray, timepts: np.ndarray, k=6):
    stream = stream[:, :3]
    X_P = []

    # Compute k-cosines for each pivot point
    for j,P in enumerate(stream):
        
        # Cannot consider edge points as central k's
        if (j - k < 0) or (j + k >= stream.shape[0]):
            continue

        P_a = stream[j - k]
        P_c = stream[j + k]

        k_cos = np.dot(P_a, P_c) / (np.linalg.norm(P_a) * np.linalg.norm(P_c))
        k_cos = max(min(k_cos, 1), -1)
        X_P.append(180 - (math.acos(k_cos) * (180/np.pi)))

    # Detect pivot points
    mu = np.mean(X_P)
    sig = np.std(X_P)

    for i in range(k):
        
        X_P.insert(0, mu)
        X_P.append(mu)

    X_P = np.array(X_P)

    F_c = [1 if x_P > mu + sig else 0 for x_P in X_P]

    j = 0
    for i in range(1, len(F_c)):

        if F_c[i] == 1 and F_c[i-1] == 0:
            j += 1
        elif sum(F_c[i:i+k]) == 0 and j != 0:
            ind = math.floor(j/2)
            F_c[i-j:i] = [0] * j
            F_c[i-ind] = 1
            j = 0
        elif j != 0:
            j += 1

    st = np.insert(timepts[[s == 1 for s in F_c]], 0, min(timepts))

    return F_c, st

strokes, stroke_times = get_strokes(data['pose_mastoidectomy_drill'][()], data['time'][()])

In [4]:
def stats_per_stroke(stroke_arr: np.ndarray):
    
    mean = np.mean(stroke_arr)
    med_ = np.median(stroke_arr)
    max_ = np.max(stroke_arr)

    return mean, med_, max_

In [5]:
def stroke_force(strokes: np.ndarray, stroke_times: np.ndarray,
                force_stream: np.ndarray, force_times: np.ndarray):

    avg_stroke_force = []
    for i in range(sum(strokes)):

        stroke_mask = [ft >= stroke_times[i] and ft < stroke_times[i+1] for ft in force_times]
        stroke_forces = np.linalg.norm(force_stream[stroke_mask], axis=1)
        avg_stroke_force.append(np.mean(stroke_forces))

    return np.array(avg_stroke_force)
    

stroke_force(strokes, stroke_times, force['wrench'][()], force['time_stamp'][()])

array([0.        , 0.        , 0.09622728])

In [36]:
def stroke_length(strokes: np.ndarray, stream: np.ndarray):

    stream = stream[:, :3]
    
    lens = []
    inds = np.insert(np.where(strokes == 1), 0, 0)
    for i in range(sum(strokes)):
        
        stroke_len = 0
        curr_stroke = stream[inds[i]:inds[i+1]]
        for j in range(1, len(curr_stroke)):

            stroke_len += np.linalg.norm(curr_stroke[j-1] - curr_stroke[j])

        lens.append(stroke_len)

    return np.array(lens)

stroke_length(np.array(strokes), data['pose_mastoidectomy_drill'][()])

array([3.35566017e-06, 1.98011122e-02, 2.20216018e-02])

In [7]:
def bone_removal_rate(strokes: np.ndarray, stroke_times: np.ndarray,
                        stream: np.ndarray, voxel_times: np.ndarray):
    
    vox_rm = []
    for i in range(sum(strokes)):

        stroke_voxels = [vt >= stroke_times[i] and vt < stroke_times[i+1] for vt in voxel_times]
        vox_rm.append(sum(stroke_voxels))

    rate = np.divide(np.array(vox_rm), stroke_length(strokes, stream))

    return rate

bone_removal_rate(strokes, stroke_times, data['pose_mastoidectomy_drill'][()], v_rm['time_stamp'][()])

array([0.0000000e+00, 0.0000000e+00, 2.9640957e+08])

In [8]:
def procedure_duration(timepts: np.ndarray):
    return (max(timepts) - min(timepts))

procedure_duration(data['time'][()])

36.614747524261475

In [26]:
def drill_orientation(stream: np.ndarray, timepts: np.ndarray,
                      voxels: np.ndarray, voxel_times: np.ndarray):
    
    print(len(voxel_times))
    print('howdy')
    # for t in timepts:
    #     ind = np.argmin(np.abs(voxel_times - t))
    #     if np.isclose(np.abs(voxel_times[ind] - t), 0):
    #         if sum([v == voxel_times[ind] for v in voxel_times]) >= 3:
    #             print(ind)
    #             print('howdy')
    #             return
    
    for i in range(len(voxel_times)-1,-1,-1):
        print(i)
        if sum([v == voxel_times[i] for v in voxel_times]) >= 3:
            print(ind)
            print('howdy')
            return

drill_orientation(data['pose_mastoidectomy_drill'][()], data['time'][()], v_rm['voxel_removed'][()], v_rm['time_stamp'][()])

13657
howdy
13656
13655
13654
13653
13652
13651
13650
13649
13648
13647
13646
13645
13644
13643
13642
13641
13640
13639
13638
13637
13636
13635
13634
13633
13632
13631
13630
13629
13628
13627
13626
13625
13624
13623
13622
13621
13620
13619
13618
13617
13616
13615
13614
13613
13612
13611
13610
13609
13608
13607
13606
13605
13604
13603
13602
13601
13600
13599
13598
13597
13596
13595
13594
13593
13592
13591
13590
13589
13588
13587
13586
13585
13584
13583
13582
13581
13580
13579
13578
13577
13576
13575
13574
13573
13572
13571
13570
13569
13568
13567
13566
13565
13564
13563
13562
13561
13560
13559
13558
13557
13556
13555
13554
13553
13552
13551
13550
13549
13548
13547
13546
13545
13544
13543
13542
13541
13540
13539
13538
13537
13536
13535
13534
13533
13532
13531
13530
13529
13528
13527
13526
13525
13524
13523
13522
13521
13520
13519
13518
13517
13516
13515
13514
13513
13512
13511
13510
13509
13508
13507
13506
13505
13504
13503
13502
13501
13500
13499
13498
13497
13496
13495
13494
13493
1349

KeyboardInterrupt: 